In [73]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge

In [68]:
train_data = pd.read_table('../../datasets/zhengqi/zhengqi_train.txt')
test_data = pd.read_table('../../datasets/zhengqi/zhengqi_test.txt')

In [69]:
train_data.describe()

,V0,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V29,V30,V31,V32,V33,V34,V35,V36,V37,target
count,2888.000000,2888.000000,2888.000000,2888.000000,2888.000000,2888.000000,2888.000000,2888.000000,2888.000000,2888.000000,...,2888.000000,2888.000000,2888.000000,2888.000000,2888.000000,2888.000000,2888.000000,2888.000000,2888.000000,2888.000000
mean,0.123048,0.056068,0.289720,-0.067790,0.012921,-0.558565,0.182892,0.116155,0.177856,-0.169452,...,0.097648,0.055477,0.127791,0.020806,0.007801,0.006715,0.197764,0.030658,-0.130330,0.126353
std,0.928031,0.941515,0.911236,0.970298,0.888377,0.517957,0.918054,0.955116,0.895444,0.953813,...,1.061200,0.901934,0.873028,0.902584,1.006995,1.003291,0.985675,0.970812,1.017196,0.983966
min,-4.335000,-5.122000,-3.420000,-3.956000,-4.742000,-2.182000,-4.576000,-5.048000,-4.692000,-12.891000,...,-2.912000,-4.507000,-5.859000,-4.053000,-4.627000,-4.789000,-5.695000,-2.608000,-3.630000,-3.044000
25%,-0.297000,-0.226250,-0.313000,-0.652250,-0.385000,-0.853000,-0.310000,-0.295000,-0.159000,-0.390000,...,-0.664000,-0.283000,-0.170250,-0.407250,-0.499000,-0.290000,-0.202500,-0.413000,-0.798250,-0.350250
50%,0.359000,0.272500,0.386000,-0.044500,0.110000,-0.466000,0.388000,0.344000,0.362000,0.042000,...,-0.023000,0.053500,0.299500,0.039000,-0.040000,0.160000,0.364000,0.137000,-0.185500,0.313000
75%,0.726000,0.599000,0.918250,0.624000,0.550250,-0.154000,0.831250,0.782250,0.726000,0.042000,...,0.745250,0.488000,0.635000,0.557000,0.462000,0.273000,0.602000,0.644250,0.495250,0.793250
max,2.121000,1.918000,2.828000,2.457000,2.689000,0.489000,1.895000,1.918000,2.245000,1.335000,...,4.580000,2.689000,2.013000,2.395000,5.465000,5.110000,2.324000,5.238000,3.000000,2.538000


In [70]:
y_train = train_data['target'].values
X_train = train_data.drop('target', axis=1).values
X_test = test_data.values

min_max_scaler = MinMaxScaler()
X_train = min_max_scaler.fit_transform(X_train)
X_test = min_max_scaler.fit_transform(X_test)

In [71]:
##### xgb
xgb_params = {'eta': 0.1, 'max_depth': 4, 'subsample': 0.6, 'colsample_bytree': 0.8, 
          'objective': 'reg:linear', 'eval_metric': 'rmse', 'silent': True, 'nthread': 16}

# reg:linear 线性回归；reg:logistic 逻辑回归

folds = KFold(n_splits=5, shuffle=True, random_state=2019)
oof_xgb = np.zeros(len(train_data))
predictions_xgb = np.zeros(len(test_data))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
    print("fold n°{}".format(fold_+1))
    trn_data = xgb.DMatrix(X_train[trn_idx], y_train[trn_idx])
    val_data = xgb.DMatrix(X_train[val_idx], y_train[val_idx])

    watchlist = [(trn_data, 'train'), (val_data, 'valid_data')]
    clf = xgb.train(dtrain=trn_data, num_boost_round=20000, evals=watchlist, early_stopping_rounds=200, verbose_eval=100, params=xgb_params)
    oof_xgb[val_idx] = clf.predict(xgb.DMatrix(X_train[val_idx]), ntree_limit=clf.best_ntree_limit)
    predictions_xgb += clf.predict(xgb.DMatrix(X_test), ntree_limit=clf.best_ntree_limit) / folds.n_splits

print("CV score: {:<8.8f}".format(mean_squared_error(oof_xgb, y_train)))

fold n°1
[0]	train-rmse:0.962224	valid_data-rmse:0.981312
Multiple eval metrics have been passed: 'valid_data-rmse' will be used for early stopping.

Will train until valid_data-rmse hasn't improved in 200 rounds.
[100]	train-rmse:0.213893	valid_data-rmse:0.332196
[200]	train-rmse:0.153767	valid_data-rmse:0.327006
[300]	train-rmse:0.114048	valid_data-rmse:0.325163
[400]	train-rmse:0.088236	valid_data-rmse:0.321667
[500]	train-rmse:0.068231	valid_data-rmse:0.320212
[600]	train-rmse:0.053331	valid_data-rmse:0.320307
[700]	train-rmse:0.041446	valid_data-rmse:0.320105
[800]	train-rmse:0.03242	valid_data-rmse:0.319863
[900]	train-rmse:0.025701	valid_data-rmse:0.320174
[1000]	train-rmse:0.02062	valid_data-rmse:0.320053
Stopping. Best iteration:
[810]	train-rmse:0.031623	valid_data-rmse:0.319828

fold n°2
[0]	train-rmse:0.977985	valid_data-rmse:0.904619
Multiple eval metrics have been passed: 'valid_data-rmse' will be used for early stopping.

Will train until valid_data-rmse hasn't improved 

In [79]:
##### ridge
reg = Ridge(alpha = .5)

folds = KFold(n_splits=5, shuffle=True, random_state=2019)
oof_reg = np.zeros(len(train_data))
predictions_reg = np.zeros(len(test_data))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
    print("fold n°{}".format(fold_+1))

    clf = reg.fit(X_train[trn_idx], y_train[trn_idx])
    oof_reg[val_idx] = clf.predict(X_train[val_idx])
    print("fold n°{}".format(mean_squared_error(oof_reg[val_idx], y_train[val_idx])))
    predictions_reg += clf.predict(X_test) / folds.n_splits

print("CV score: {:<8.8f}".format(mean_squared_error(oof_reg, y_train)))

fold n°1
fold n°0.10888544804801513
fold n°2
fold n°0.13954093483790903
fold n°3
fold n°0.0943499326025632
fold n°4
fold n°0.11501970468666108
fold n°5
fold n°0.10329170773049902
CV score: 0.11221967


In [84]:
series = pd.Series(predictions_reg)
series.to_csv('../../datasets/zhengqi/submit.txt', index=None, header=None)